In [ ]:
RANDOM_SEED = 43

# Dataset

In [ ]:
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, DataLoader
import matplotlib.pyplot as plt

In [ ]:
image_size = (64, 64)
batch_size = 64

In [ ]:
image_transforms = transforms.Compose(
    [
        transforms.Resize(image_size),
        transforms.ToTensor()
    ]
)

A feladat megoldását 2 külön adathalmazzal is szeretnénk megtenni:
* első és fontosabb a celeba dataset, amely celebek arcait tartalmazza előfeldolgozottan (cropped, aligned)
* második a danbooru dataset, amely anime karakterek arcait tartalmazza
* (opcionálisan egy kevert adathalmazt is szeretnénk tesztelni, hogy milyen eredményeket tudunk kapni)

Az adathalmazokat előre letöltöttük és kicsomagoltuk a tömörített fájlokat, majd így egy volume segítségével kerülnek a containerhez felcsatolásra

Mivel képgenerálásról beszélünk, a tesztelési fázis nem teljesen jelent egyértelmű feladatot
Ennek ellenére felkészülünk training, validation és test dataloaderekkel is, melyeknek bemenete a random 8:1:1 arányban felosztott adathalmaz. Kimenetük pedig egy batch_size-onként "adagolt" adathalmaz a modellünknek, image_size formájú 3 csatornás (RGB) Tensorokként

In [ ]:
import os 

data_path = os.environ['CELEBA']
# data_path = os.environ['DANBOORU']

In [ ]:
generator = torch.Generator().manual_seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)
torch.backends.cudnn.deterministic = True

In [ ]:
data = ImageFolder(root=data_path, transform=image_transforms)
train_data, val_data, test_data = random_split(data, [0.8, 0.1, 0.1], generator=generator)

train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=1, pin_memory=True)
val_dataloader = DataLoader(val_data, batch_size=batch_size, shuffle=True, num_workers=1, pin_memory=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=1, pin_memory=True)

In [ ]:
for x, _ in train_dataloader:
    print(x[0].shape)
    plt.imshow(x[0].permute(1, 2, 0))
    break

# Model

# Training

# Evaluation